Didn't miss writing a notebook for statistics that much :|

First things first we need to change the extensions of the matches to txt from tcx (the export format of choice of the garmin connect app) but first some imports

In [46]:
import os
import numpy as np
import pandas as pd
import csv
import chess
import chess.pgn
import chess.engine
from stockfish import Stockfish


In [47]:
def change_ext(cartella, old_ext, new_ext):
    for root, dirs, files in os.walk(cartella):
        # print(os.getcwd())
        for f in files:
            filename = os.fsdecode(f)
            
                
            if filename.endswith(old_ext):
                name, ext = os.path.splitext(filename)
                
                new_name = name + new_ext
                jpath = os.path.join(root,new_name)
                os.rename(os.path.join(root,filename), jpath)
                print(f"renamed {os.path.join(root,filename)} to {jpath}")

change_ext("games", ".tcx", ".txt")

Now we have to import the data from the [Portable Game Notation or pgn](https://en.wikipedia.org/wiki/Portable_Game_Notation)
and read the heartbeats of both white and black (in the games where they are both available) in order to create a 3d dataframe for every game and then to visualize the data we are working with. To process the portable game notation we will use the following cell of code.

In [56]:
### UTILS FOR READING THE ACTIVITIES

def calctime(start,curr):
    c = [int(x) for x in curr.split(":")]
    s = [int(x) for x in start.split(":")]
    return ((c[0] - s[0]) * 60 + (c[1] - s[1])) * 60 + (c[2] - s[2])

def read_activity(game):
    with open(game) as f:
        lines = f.readlines()
        hr = []
        time = []
        for s in lines:
            if "<Value>" in s:
                hr.append(s.strip()[s.strip().find(">") + 1:s.strip().rfind("<")])
            if "<Time>" in s:
                s = s.strip()[3:]
                if not time:
                    start = s.strip()[s.strip().find("T") + 1:s.strip().rfind(".")]
                    
                time.append(calctime(start, s.strip()[s.strip().find("T") + 1:s.strip().rfind(".")]))
        
        #print(f"hr is {hr}, and time is {time}")
        hr = hr[2:]
        if len(time) - 1 != time[-1]:
            diz = {time[i]: hr[i] for i in range(min(len(hr), len(time)))}
            last_battito = 0
            res = []
            keys = list(diz.keys())
            
            
            for s in range(time[-1] + 1):
                if s in diz:
                    res.append((diz[s], s))
                    last = (diz[s], s)
                    indice_s = keys.index(s)
                    if indice_s +1 < len(keys):
                        next_s = keys[indice_s + 1]
                        next = (diz[keys[indice_s + 1]], next_s)
                    
                    
                else:
                    total_gap = s - last[1] + next[1] - s
                    weight_next = (s-last[1]) * 100 / total_gap
                    weight_last = 100 - weight_next 
                    est_battito = (weight_last * float(last[0]) + weight_next * float(next[0])) / 100
                    res.append((round(est_battito), s))
            return res
                
        
        return list(zip(hr,time))
        
read_activity("games/partita_0816-7/w_activity_9420267252_tom.txt")

[('82', 0),
 ('82', 1),
 ('83', 2),
 ('83', 3),
 ('85', 4),
 ('86', 5),
 ('86', 6),
 ('86', 7),
 ('85', 8),
 ('85', 9),
 ('84', 10),
 ('86', 11),
 ('86', 12),
 ('88', 13),
 ('89', 14),
 ('88', 15),
 ('87', 16),
 ('86', 17),
 ('84', 18),
 ('84', 19),
 ('84', 20),
 ('85', 21),
 ('85', 22),
 ('86', 23),
 ('85', 24),
 ('84', 25),
 ('84', 26),
 ('85', 27),
 ('85', 28),
 ('85', 29),
 ('86', 30),
 ('86', 31),
 ('87', 32),
 ('88', 33),
 ('88', 34),
 ('88', 35),
 ('88', 36),
 ('88', 37),
 ('87', 38),
 ('87', 39),
 ('87', 40),
 ('87', 41),
 ('88', 42),
 ('88', 43),
 ('89', 44),
 ('90', 45),
 ('91', 46),
 ('91', 47),
 ('91', 48),
 ('91', 49),
 ('91', 50),
 ('91', 51),
 ('91', 52),
 ('91', 53),
 ('91', 54),
 ('91', 55),
 ('90', 56),
 ('90', 57),
 ('90', 58),
 ('90', 59),
 ('91', 60),
 ('91', 61),
 ('90', 62),
 ('89', 63),
 ('89', 64),
 ('89', 65),
 ('89', 66),
 ('89', 67),
 ('88', 68),
 ('88', 69),
 ('88', 70),
 ('88', 71),
 ('88', 72),
 ('88', 73),
 ('89', 74),
 ('90', 75),
 ('92', 76),
 ('94', 7

In [49]:
### UTILS FOR READING THE PGN

#this method takes as input the board and the last move so to update the board and compute the evaluation
def get_evaluation(board,move):

    stockfish = Stockfish("/usr/local/Cellar/stockfish/15/bin/stockfish")     #if no parameter passed, takes as input the executable file called stockfish from the same directory, if found
    board.push_san(move)        #insert the move in the board
    stockfish.set_fen_position(board.fen())  #bpard.fen() applies a PGN to FEN conversion
    engine = chess.engine.SimpleEngine.popen_uci("stockfish")
    info = engine.analyse(board, chess.engine.Limit(time=0.1))     #gets the infos of the game
    
    #if the position on the board is not a mate the score is in centipawns, then it's divided by 100 to get the actual score
    #Gets the score from White’s point of view
    if chess.engine.PovScore.is_mate(info['score']):
        eval = chess.engine.PovScore.white(info['score'])
    else:
        eval = int(str(chess.engine.PovScore.white(info['score'])))/100

    return board, eval      #returns the board updated and the evaluation of the position

In [50]:
def get_timestamps(file):
    timestamps = []
    file = file[:-3] + "txt"
    with open(file, "r") as f:
        file_read = f.read()
        lista = file_read.split("%")
        
        
        for el in lista:
            if el[0:4] == "time":
                num = ""
                flag = False
                for i  in range(len(el)):
                    while el[i].isdigit():
                        num += el[i]
                        i += 1
                        flag = True
                    if flag:
                        timestamps.append(int(num)/10)
                        break
    return timestamps
            
get_timestamps("/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/games/partita_0816-7/tommidim_vs_roccot01_2022.08.16.txt")

[0.1,
 0.1,
 3.4,
 0.9,
 13.6,
 1.9,
 2.8,
 9.8,
 3.5,
 11.4,
 13.7,
 9.5,
 13.0,
 19.4,
 12.9,
 11.3,
 2.5,
 1.7,
 17.7,
 5.5,
 10.1,
 2.7,
 0.6,
 4.8,
 21.2,
 4.4,
 0.1,
 10.4,
 2.5,
 22.1,
 10.8,
 12.1,
 23.1,
 6.5,
 1.6,
 20.3,
 7.7,
 49.1,
 27.4,
 10.1,
 3.5,
 16.7,
 6.2,
 8.7,
 37.7,
 34.0,
 49.1,
 9.7,
 26.9,
 29.2,
 11.8,
 1.4,
 12.9,
 22.8,
 17.7,
 17.7,
 12.9,
 20.9,
 54.3,
 12.6,
 6.0,
 1.6,
 0.4,
 1.0,
 0.4,
 26.9,
 3.7,
 7.1,
 25.2,
 15.9,
 42.5,
 7.4,
 15.1,
 16.0,
 10.7,
 21.5,
 7.9,
 13.1,
 21.0,
 1.7,
 2.0,
 22.3,
 1.1,
 19.5,
 6.0,
 18.0,
 19.3,
 12.7,
 0.7,
 3.9,
 2.0,
 6.2,
 3.8,
 1.3,
 3.1,
 3.2,
 7.7,
 1.5,
 0.1,
 1.1]

In [52]:
#read_pgn takes as input the name of the file pgn containing the game and returns a dataframe
def read_pgn(game_name): 
    old = False
    if "05.29" in game_name or "05.25" in game_name or "clk" in game_name:
        old = True
    #opens and reads the pgn file
    if not old:
        timestamps = get_timestamps(game_name)
    pgn = open(game_name) 
    
    game = chess.pgn.read_game(pgn)
    
                                   
    array = []      #initialize the array that will contain the data
    index_values = [] 
    i=0
    time_white = -0.1      #to keep track of the time for each player
    time_black = 0      #to keep track of the time for each player
    board = chess.Board()       #initialize the chessboard

    #iterate through the moves
    for node in game.mainline():        
        
        move = str(node.move)       #represents the last move
        board,evaluation = get_evaluation(board,move)       #the get_evaluation method returns the updated board and the evaluation of the current position
        
        
        if i%2 == 0:        #if the white is moving we update his time
            index_values.append("white")
            if old:
                time_white = 600 - node.clock()
            else:
                time_white += timestamps[i]
                array.append([str(node.move),round(time_white,4),round(time_white+time_black,4),evaluation])
            
           
        else:           #if the black is moving we update his time
            index_values.append("black") 
            if old:
                time_black = 600 - node.clock()
            else:
                time_black += timestamps[i]
                array.append([str(node.move),round(time_black,4),round(time_white+time_black,4),evaluation])

        #evaluation = get_evaluation(board,move)
        #we append an element in the array (a row in the dataframe) with move, time the player used, total time since the game started and evaluation of the position
        # if str(evaluation)[:2] == "#+":
        #     evaluation = 15
        # elif str(evaluation)[:2] == "#-":
        #     evaluation = -15
        if old:
            array.append([str(node.move),round(600 - node.clock(),4),round(time_white+time_black,4),evaluation])   
        i+=1
    
    column_values = ['move','time','real time','evaluation'] #create the columns names
    array = array      #convert our array to a numpy array
    res = []
    j = 0
    #df = pd.DataFrame(data = array, index = index_values, columns = column_values) 
    
    tub = 0
    tuw = 0
    for i in range(int(array[-1][2]) + 1):
        mosse = []
        
        while float(array[j][2]) <= float(i):
            pos = array[j][3]
            mosse.append(array[j][0])
            if j % 2 == 0:
                tuw = array[j][1]
            elif j % 2 != 0:
                tub = array[j][1]
            j += 1
            if j > len(array):
                break
        res.append([i, pos, (tuw,tub), mosse])
        
            
        
    #create a dataframe
    print(array)
    return res


#/Users/tommasodimario/Documents/GitHub/statistics-101/project/src/games/partite_confronto/partita_timestamp.pgn

In [62]:
file = open("prova.csv", "w")

writer = csv.writer(file)

columns = ["Time", "Game", "Player", "Statistics", "Value"]
writer.writerow(columns)

#pseudo code
# to do create queste cose così da poter iterare in questa roba della morte e creare un dataset di questo tipo
# https://raw.githubusercontent.com/QuantEcon/lecture-python/master/source/_static/lecture_specific/pandas_panel/realwage.csv
# sembra particolarmente greve ma se n è il numero di statistiche gira in ~20*2*3 * n quindi è un n^4 molto tranquillo
print(os.getcwd())



gamecount = 0
count = 0   
for root,dirs, files in os.walk("games"):
    if str(root)  != "games":
        partita = f"Game{gamecount}"
        gamecount += 1
        if gamecount > 1: # così fa solo le prime tre partite
            break
    for file in files:
        #qui siamo dentro i file di una singola partita
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".txt") and (filename[0] == "w" or filename[0] == "b"): # abbiamo un carissimo file delle attività
        #leggiamo i battiti
            player = filename[0]
            activity = read_activity(f"{str(root)}/{filename}")
            # As explained in the markdown above on our strumentation TODO our smartwatches recorded the heartbeats at different time so we smoothed it out
            for hr, time in activity:
                row = [count,int(time), partita, player, "HR", int(hr)]
                writer.writerow(row)
                count +=1
        elif filename.endswith(".pgn"):
            
            pgn = read_pgn(f"{str(root)}/{filename}")
            for second in range(len(pgn)):
                ev = pgn[second][1]
                try:
                    ev = float(pgn[second][1])
                except:
                    ev = 20
                row = [count,second, partita, "w", "Evaluation", ev]
                row = [count,second, partita, "b", "Evaluation", -ev]
                
                writer.writerow(row)
                count +=1
                    
            for second in range(len(pgn)):
                for player in ["b", "w"]:       
                    p = 0 if player == "w" else 1
                    print(f" at time {second} the pgn is {pgn[second]}")
                    row = [count, second, partita, player, "TimeUsed", int(pgn[second][2][p])]
                    writer.writerow(row)
                    count +=1
# 4.



/Users/tommasodimario/Documents/GitHub/statistics-101/project/src
.DS_Store
.DS_Store
roccot01_vs_tommidim_2022.08.16.txt
b_activity_9418334461_tom.txt
roccot01_vs_tommidim_2022.08.16.pgn
[['e2e4', 0.0, 0.0, 0.38], ['e7e5', 0.1, 0.1, 0.42], ['b1c3', 11.1, 11.2, 0.1], ['g8f6', 5.4, 16.5, 0.18], ['f1c4', 19.2, 24.6, 0.08], ['c7c6', 35.4, 54.6, 0.98], ['g1f3', 50.2, 85.6, 0.07], ['d7d5', 41.0, 91.2, 0.62], ['e4d5', 71.8, 112.8, 0.11], ['c6d5', 42.2, 114.0, 0.86], ['c4b5', 85.1, 127.3, 0.74], ['c8d7', 51.6, 136.7, 0.61], ['f3e5', 89.7, 141.3, 0.96], ['d7b5', 59.0, 148.7, 1.21], ['c3b5', 95.4, 154.4, 1.14], ['d8e7', 60.9, 156.3, 2.11], ['d1e2', 120.9, 181.8, 2.37], ['b8c6', 154.5, 275.4, 5.0], ['d2d4', 131.6, 286.1, 1.68], ['a7a6', 159.3, 290.9, 1.89], ['e5c6', 220.1, 379.4, 1.81], ['e7e2', 213.4, 433.5, 2.02], ['e1e2', 222.1, 435.5, 1.95], ['a6b5', 214.8, 436.9, 1.59], ['c6e5', 235.8, 450.6, 1.63], ['f8d6', 226.4, 462.2, 1.98], ['h1e1', 242.2, 468.6, 1.87], ['e8g8', 229.4, 471.6, 1.68], ['

In [60]:

# Display 6 columns for viewing purposes
pd.set_option('display.max_columns', 7)

# Reduce decimal points to 2
# pd.options.display.float_format = '{:,.2f}'.format

realwage = pd.read_csv("prova.csv")
realwage = realwage.pivot_table(values='Value',
                                index='Time',
                                columns=['Game', 'Player', 'Statistics'])
realwage.head(15)

Game            Game0                              
Player              b                    w         
Statistics Evaluation    HR TimeUsed    HR TimeUsed
Time                                               
0               -0.38 86.00     0.00 75.00     0.00
1               -0.50 87.00     0.00 77.00     0.00
2               -0.50 87.00     0.00 77.00     0.00
3               -0.50 88.00     0.00 78.00     0.00
4               -0.50 89.00     0.00 78.00     0.00
5               -0.50 89.00     0.00 78.00     0.00
6               -0.50 88.00     0.00 78.00     0.00
7               -0.50 88.00     0.00 79.00     0.00
8               -0.50 88.00     0.00 79.00     0.00
9               -0.50 88.00     0.00 79.00     0.00
10              -0.50 89.00     0.00 79.00     0.00
11              -0.50 89.00     0.00 80.00     0.00
12              -0.16 89.00     0.00 80.00    11.00
13              -0.16 89.00     0.00 82.00    11.00
14              -0.16 91.00     0.00 83.00    11.00

In [ ]:
realwage["Game0"]

Player        Black      White         
Statistics TimeUsed Evaluation TimeUsed
Time                                   
0              0.00       0.38     0.00
1              0.00       0.38     0.00
2              0.00       0.38     0.00
3              2.00       0.55     0.00
4              2.00       0.37     1.00
...             ...        ...      ...
156           58.00       9.35    87.00
157           58.00       9.35    87.00
158           58.00       9.35    87.00
159           58.00       9.35    87.00
160           58.00       9.35    87.00

[161 rows x 3 columns]